# CatBoost 详尽介绍

## 1. 背景介绍

## 2. 核心特点

## 3. 工作原理

### 3.1 梯度提升树（GBDT）

### 3.2 类别特征处理

### 3.3 有序Boosting vs 普通Boosting

## 4. 常用参数说明

#### 创建Pool数据结构
train_pool = Pool( data=X, label=y, cat_features=cat_features)

#### 初始化模型
model = CatBoostClassifier(iterations=200,
                           learning_rate=0.1,
                           depth=6,
                           verbose=50)

| 参数                      | 说明                                  | 常用示例值                |
| ----------------------- | ----------------------------------- | -------------------- |
| iterations              | 训练的树的数量（最大迭代次数）                     | 1000                 |
| learning\_rate          | 学习率，控制每棵树的贡献大小                      | 0.03-0.1             |
| depth                   | 树的最大深度                              | 6                    |
| loss\_function          | 损失函数，如 `Logloss` (二分类), `RMSE` (回归) | 自动推断或显式指定            |
| cat\_features           | 指定类别特征的列名或索引                        | `['city', 'gender']` |
| eval\_metric            | 评估指标，如 `AUC`, `Accuracy`            | 根据任务选取               |
| random\_seed            | 随机种子，保证结果复现                         | 42                   |
| early\_stopping\_rounds | 早停轮数，防止过拟合                          | 50                   |
| task\_type              | 训练类型，如 `'CPU'` 或 `'GPU'`            | `'CPU'` 或 `'GPU'`    |


## 5. 优缺点分析
优点

原生支持类别特征，无需预处理，使用方便。

防止目标泄漏，泛化能力强。

训练速度快，支持CPU/GPU。

多种任务支持，应用广泛。

多语言接口丰富。

缺点

参数较多，新手需要时间熟悉调参。

有序Boosting训练时序列化操作增加计算开销，单次训练时间可能稍长。

对极大规模数据集在某些场景下可能稍逊于LightGBM。

## 6. 简单使用示例（Python）

In [ ]:
from catboost import CatBoostClassifier, Pool
import pandas as pd

# 创建示例数据
data = {
    'city': ['北京', '上海', '广州', '北京', '上海'],
    'age': [25, 30, 45, 35, 40],
    'buy': [1, 0, 1, 0, 1]
}
df = pd.DataFrame(data)

X = df[['city', 'age']]
y = df['buy']

# 指定类别特征
cat_features = ['city']

# 创建Pool数据结构
train_pool = Pool(data=X, label=y, cat_features=cat_features)

# 初始化模型
model = CatBoostClassifier(iterations=200,
                           learning_rate=0.1,
                           depth=6,
                           verbose=50)  #  verbose：打印训练进度。

# 训练模型
model.fit(train_pool)

# 预测
preds = model.predict(X)
print("预测结果:", preds)


## CatBoost 常用方法概览

### 1. 模型构造与训练

### 2. 预测

### 3. 模型评估和调试

In [ ]:
.get_feature_importance(data=None, fstr_type='FeatureImportance', thread_count=-1, ...) --->  获取特征重要性。

fstr_type 可以是 FeatureImportance、ShapValues、PredictionValuesChange 等。

.get_best_score()  --->  返回验证集上的最佳分数。

.get_params()  --->  获取模型参数。

.set_params(**params)  --->  设置模型参数。

### 4. 模型保存与加载

In [ ]:
.save_model(fname, format='cbm', export_parameters=None, ...)  --->  保存模型文件。

.load_model(fname, format='cbm', ...)  --->  加载已保存的模型。


### 5. 交叉验证

In [ ]:
catboost.cv(pool, params, fold_count=3, inverted=False, shuffle=True, partition_random_seed=0, stratified=False, ...)

进行交叉验证，返回折叠结果。

## 具体示例

In [ ]:
from catboost import CatBoostClassifier, Pool, cv

# 1. 准备数据
X = [[1, 'A'], [2, 'B'], [3, 'A'], [4, 'B']]
y = [0, 1, 0, 1]
cat_features = [1]  # 第二列是类别特征

# 2. 创建模型
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6)

# 3. 训练
model.fit(X, y, cat_features=cat_features, verbose=10)

# 4. 预测
preds = model.predict(X)

# 5. 获取特征重要性
fi = model.get_feature_importance()

# 6. 保存模型
model.save_model("catboost_model.cbm")

# 7. 加载模型
model2 = CatBoostClassifier()
model2.load_model("catboost_model.cbm")

# 8. 交叉验证
pool = Pool(data=X, label=y, cat_features=cat_features)
cv_data = cv(pool, params={'iterations': 100, 'depth': 6}, fold_count=3)


# CatBoost 调参技巧详解

In [ ]:
## 1. 重要参数介绍

In [ ]:
| 参数                      | 作用                                                | 推荐调整思路                              |
| ----------------------- | ------------------------------------------------- | ----------------------------------- |
| `iterations`            | 树的总数（迭代次数）                                        | 适当增大，结合早停确定最佳迭代次数                   |
| `learning_rate`         | 学习率，控制每棵树的贡献大小                                    | 先用较大值快速训练，再减小微调                     |
| `depth`                 | 树的深度，控制模型复杂度                                      | 4\~10之间调试，防止过拟合                     |
| `l2_leaf_reg`           | L2正则化系数，防止过拟合                                     | 3\~10间调整，数值越大正则化越强                  |
| `border_count`          | 数值特征分箱数，影响精度和速度                                   | 默认128，调整对速度和精度有影响                   |
| `bagging_temperature`   | 控制随机样本采样强度，影响过拟合和泛化                               | 0\~1之间调节，越大随机性越强                    |
| `random_strength`       | 分类特征随机强度，防止过拟合                                    | 1\~20之间调节                           |
| `grow_policy`           | 树的生长策略（`SymmetricTree`, `Depthwise`, `Lossguide`） | `SymmetricTree`是默认，`Lossguide`适合大数据 |
| `early_stopping_rounds` | 早停轮数，避免过拟合                                        | 50\~100之间，根据验证集效果调整                 |


## 2. 调参流程建议

### 2.1 确定训练轮数和学习率
先设置一个较大的 iterations（如 1000~2000）。

用较小的 learning_rate（0.03~0.1）。

利用 early_stopping_rounds 让训练自动停止，找到最佳迭代次数。

### 2.2 调整树深度 depth
深度越大，模型越复杂，容易过拟合。

先从6开始尝试，调小可以防过拟合，调大提升拟合能力。

### 2.3 正则化调节
调整 l2_leaf_reg，增加可减轻过拟合。

调节 bagging_temperature 和 random_strength，提升模型泛化。

### 2.4 类别特征参数
CatBoost自动处理类别特征，但可以调节 one_hot_max_size 参数（类别数较少时用 One-Hot，类别多时用统计编码）。

注意不要对类别特征做过多手工编码。

### 2.5 选择合适的生长策略 grow_policy
默认 SymmetricTree（对称树）训练快，效果好。

对大数据或复杂任务，试试 Lossguide。

## 3. 具体调参示例（Python）

In [ ]:
from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=7,
    l2_leaf_reg=5,
    bagging_temperature=1,
    random_strength=10,
    early_stopping_rounds=100,
    grow_policy='SymmetricTree',
    verbose=100
)

train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
eval_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)

model.fit(train_pool, eval_set=eval_pool, use_best_model=True)


## 4. 其他实用建议
多次训练取平均：设置不同随机种子，多次训练，结果取平均降低波动。

交叉验证：使用 cv() 函数做 K 折交叉验证，寻找稳定的参数组合。

调参工具：结合 Optuna、Hyperopt 等自动调参库，效率更高。

GPU 加速：训练时加参数 task_type='GPU'，能大幅加快训练速度。

## 5. 调参思路总结

| 调参阶段    | 重点调整参数                                   | 目标            |
| ------- | ---------------------------------------- | ------------- |
| 初步训练    | `iterations`, `learning_rate`            | 找到合理的训练轮数和学习率 |
| 模型复杂度调整 | `depth`, `l2_leaf_reg`                   | 控制模型的拟合与泛化    |
| 泛化能力提升  | `bagging_temperature`, `random_strength` | 防止过拟合，增强鲁棒性   |
| 训练策略优化  | `grow_policy`, `early_stopping_rounds`   | 提升训练速度和防止过拟合  |
